<div style="float: left; width: 25%;">
<ul>
<img src="http://www.minesnewsroom.com/sites/default/files/wysiwyg-editor/photos/CO-Mines-logo-stacked-4C-200x235w.png" style="height: 115px;">
</ul>
</div>

<div style="float: right; width: 75%;">
<ul>
    <h1> CSCI 250 - Building a Sensor System </h1>
    <span style="color:red">
        <h2> Accelerometer </h2>
    </span>
</ul>
</div>

# Introduction

<img src="https://cdn.sparkfun.com//assets/parts/1/0/0/0/2/13926-02.jpg" width="30%" align=right>

A simple accelerometer could be constructed with a mass attached to a spring. The spring  would stretch by some amount due to Earth's gravity if oriented vertically (if it is horizontal, then there is no stretch). A spring accelerated vertically would change its length, from which we can evaluate its accelerating. Modern accelerometers are made on tiny silicon chips, but work on the same principles. Accelerometers often have 3 "springs" to measure acceleration in each direction. You can learn more about accelerometers on [this page](https://en.wikipedia.org/wiki/Accelerometer).

# Wiring

Our kits come with the MMA8452Q, which is the number that identifies the specific accelerometer we are using. It's useful to read through the [datasheet](https://cdn.sparkfun.com/datasheets/Sensors/Accelerometers/MMA8452Q-rev8.1.pdf) for this sensor, as there is lots of information we need in order to use the sensor it. You can also find it under the documents tab of [the SparkFun accelerometer](https://www.sparkfun.com/products/13926) page or just Googling "MMA8452Q datasheet". The most important information is summarized in this notebook in order to get the accelerometer working.

The accelerometer we are using communicates over I<sup>2</sup>C (explained in its own [notebook](h_I2C.ipynb)), which requires 2 wires for communication (**SDA** for serial data, and **SCL** for serial clock), and 2 wires for power (**3.3V** and **GND**). You can connect the pins of the accelerometer to the RPi using jumper wires in order to setup communication and power the chip; you'll notice that they have the exact same names. You can ignore pins I1 and I2.

<img src="https://www.dropbox.com/s/8tvxlasu3csxemj/wiring_accelerometer.PNG?raw=1" width="60%" align="left">

<img src="https://www.dropbox.com/s/e4c87okt6vc4yx1/circuit_accelerometer.jpg?raw=1" width="35%" align="right">

**N.B.**: You may find it annoying to rotate your entire breadboard to get different measurements, so you could make use of the M/F jumper wires to freely move the accelerometer without moving the breadboard.

# I<sup>2</sup>C interface

The accelerometer communicates with the RPi over the protocol called [Inter-Integrated Circuit (I<sup>2</sup>C)](https://en.wikipedia.org/wiki/I%C2%B2C) (pronounced I-squared-C). In order to use this protocol, we need to enable the I<sup>2</sup>C interface on the RPi: go to the RPi Configuration from Preferences, then on the Interfaces tab for I<sup>2</sup>C click enable, then OK (you should have already done this from setup day). This will allow us to actually use I<sup>2</sup>C.

It is not important to know all the details of how I<sup>2</sup>C works to use it, so this notebook contains just an abbreviated explanation. The details of the protocol are given in the [I2C notebook](h_I2C.ipynb).

>All of the devices are synced together by the clock pin of the master device, denoted by SCL for **serial clock**; this clock signal is simply a square wave that repeats. Data from all devices are then sent over the data wire, denoted SDA for **serial data**. The data sent across this wire are synced up with the clock, where a high voltage on the data line indicates a 1 bit, and a low voltage indicates a 0 bit. 

Every subordinate device has an address that is assigned when the chip is manufactured; the master device is informed of the addresses by the programmer. The first data sent out by the master is the address of the device; all of the subordinate devices read the address to see if the master is communicating with them. The master can then send data to the appropriate subordinate device, and all other devices ignore these data. Once the master stops sending data, the subordinate device can send its own data to the master through the SDA wire. 

<img src="https://cdn.sparkfun.com/assets/6/4/7/1/e/51ae0000ce395f645d000000.png" width="90%">


# I<sup>2</sup>C reader
For ease of use, we have created a module with a function that enables data collection from sensors connected via I<sup>2</sup>C. The function is called `readACC()` and it is included in the module `i2cUtils.py` (download from Canvas). If you want to learn more about how this function is derived, look in the [I2C notebook](h_I2C.ipynb) for details. 

You can import `i2cUtils` like this:

In [1]:
import i2cUtils as ic

# Testing
Call the `readACC()` function to check that it returns 3 numbers representing acceleration in m/s$^2$. The returned values come in the form of a tuple:

In [3]:
ax,ay,az = ic.readACC()
print(ax,ay,az)

-0.5077441406250001 0.42152343750000004 9.608818359375


The following code cell allows you to collect and graph data from the accelerometer.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import i2cUtils as ic

nDATA = 2000
xACC = np.zeros(nDATA,dtype='float')
yACC = np.zeros(nDATA,dtype='float')
zACC = np.zeros(nDATA,dtype='float')
tACC = np.zeros(nDATA,dtype='float')

ti = time.time()
for i in range(nDATA):

    # read data from accelerometer
    xACC[i],yACC[i],zACC[i] = ic.readACC()
    
    # get the time
    tACC[i] = time.time() - ti
    
plt.plot(tACC,xACC,'r');
plt.plot(tACC,yACC,'g');
plt.plot(tACC,zACC,'b');

plt.show()

# Debugging

Useful debugging tips for any time during the semester!

* I get error messages when I run my code.
    * There's probably an issue with your code, not any of the components! Read the error message and try to understand what it's telling you in order to fix it. If you're not sure, ask someone to help you!
* The accelerometer always returns 0.
    * You're probably having an issue communicating with the sensor, which would most likely come from a wiring problem. Double check your wiring with the accelerometer notebook to make sure the connections are correct. Also double check your code for any bugs.
    * If you can't figure out the problem, raise your hand and we'll come help you.
* I get funky numbers from the accelerometer.
    * Make sure that you're interpreting the bits correctly and making the conversions as described in the accelerometer notebook. If you don't convert from 2's complement, you will get some very weird behavior! Double check your code for any bugs related to this.
    * If you can't find any errors, go back to basics! Recreate the circuit and run the code from this notebook to see if your accelerometer still works as expected. If it does, the problem is with your code, not the sensor.
    * If you still get funky values, try your neighbor's accelerometer. If the new one works, yours is likely broken.
    * If none of the above steps help, you may be having some other issue; raise your hand and we'll come help you.